# How can we do transfer learning?

**Background:**

The purpose of this notebook is to transform a .wav file into a spectrogram and then feed it into a well known model such as InceptionV3 for feature generation

## Libraries and File Locations

In [1]:
import sys
sys.path.append("..")

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import soundfile
from scipy import signal
import librosa
import matplotlib.pyplot as plt
import specdisplay

/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/

/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/site-packages/librosa/cache.py:36: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if self.cachedir is not None:
/home/romulo/github/python_virtual_envs/soundflux/lib/python3.6/

## Define file generation parameters

**Number of Mel-frequencies to keep in the spectrograms**

In [4]:
n_mels = 128

## Defining spectrogram transformation function

## NOTE!!

We're averaging across channels, come back and try a different approach if that's not working

In [8]:
def extract_spectrogram(samples,sample_rate,n_mels=23,n_fft=2048):
    feature_set = []
    if samples.shape[1:]:
        channels = samples.shape[1:][0]
    else:
        channels = 1
        samples = np.expand_dims(samples, axis=1)
    for i in range(channels):
        sample_channel_x = samples[:,i]
        mel_spectrogram = librosa.feature.melspectrogram(y=sample_channel_x, 
                                                     sr=sample_rate,
                                                     n_fft=n_fft, 
                                                     n_mels = n_mels
                                                        )
        decibel_spec = librosa.logamplitude(mel_spectrogram,ref_power=np.max)
        feature_set.append(decibel_spec)
    feature_set = np.array(feature_set)
    feature_set = np.mean(feature_set,axis=0)
    return feature_set

# Creating the PNG spectrograms

**How are we organizing the output files?**

According to the researchers how created the dataset, larger files were used to obtain smaller audio sample for specific classes. With that in mind, having different chuncks of the same file in the training and validation datasets would cause the model to overfit and inflate validation accuracies. As a result, the researchers split the dataset into 10 folds, keeping data originated from the same files within a fold. The recomended way of working with the dataset is to use 9 out of the 10 folds for training and one for validation. This would be repeated 10 times with each fold being held as validation - (essentially k-fold).

The accuracy score should be reported by averaging the results of the 10 different training/fold exercises.


See more on this article:
https://urbansounddataset.weebly.com/urbansound8k.html

**Folder structure**

/fold/class/filename.png

This will help when using Kera's ImageDataGenerator.flow_from_directory api to train and test the data

In [11]:
urban_sounds_folder = "/media/romulo/6237-3231/urban_sound_challenge/"

In [12]:
metadata = pd.read_csv(urban_sounds_folder+'metadata/UrbanSound8K.csv')

In [13]:
for index, row in metadata[4986::].iterrows():
    print("Transforming file {}".format(index))
    
    if not os.path.exists(urban_sounds_folder+'spectrograms/fold' + str(row['fold'])):
        os.makedirs(urban_sounds_folder+'spectrograms/fold' + str(row['fold']))

    if not os.path.exists(urban_sounds_folder+'spectrograms/fold' + str(row['fold'])+ "/" + row['class']):
        os.makedirs(urban_sounds_folder+'spectrograms/fold' + str(row['fold'])+ "/" + row['class'])

    # Convert to log scale (dB). We'll use the peak power as reference.
    y, sr = soundfile.read(urban_sounds_folder+"audio/fold" + str(row['fold'])+ "/" + str(row['slice_file_name']))
    log_s = extract_spectrogram(y,sr,n_mels=n_mels,n_fft=2048)
    #new plot
    fig = plt.figure(figsize=(12,4))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)

    #getting spectrogram
    specdisplay.specshow(log_s, sr=sr, x_axis='time', y_axis='mel')

    #Saving PNG
    plt.savefig(urban_sounds_folder+'spectrograms/fold' + str(row['fold'])+ "/" + row['class'] + 
                '/' + row['slice_file_name'] + '.png')
    plt.close()

Transforming file 4986
Transforming file 4987
Transforming file 4988
Transforming file 4989
Transforming file 4990
Transforming file 4991
Transforming file 4992
Transforming file 4993
Transforming file 4994
Transforming file 4995
Transforming file 4996
Transforming file 4997
Transforming file 4998
Transforming file 4999
Transforming file 5000
Transforming file 5001
Transforming file 5002
Transforming file 5003
Transforming file 5004
Transforming file 5005
Transforming file 5006
Transforming file 5007
Transforming file 5008
Transforming file 5009
Transforming file 5010
Transforming file 5011
Transforming file 5012
Transforming file 5013
Transforming file 5014
Transforming file 5015
Transforming file 5016
Transforming file 5017
Transforming file 5018
Transforming file 5019
Transforming file 5020
Transforming file 5021
Transforming file 5022
Transforming file 5023
Transforming file 5024
Transforming file 5025
Transforming file 5026
Transforming file 5027
Transforming file 5028
Transformin

Transforming file 5343
Transforming file 5344
Transforming file 5345
Transforming file 5346
Transforming file 5347
Transforming file 5348
Transforming file 5349
Transforming file 5350
Transforming file 5351
Transforming file 5352
Transforming file 5353
Transforming file 5354
Transforming file 5355
Transforming file 5356
Transforming file 5357
Transforming file 5358
Transforming file 5359
Transforming file 5360
Transforming file 5361
Transforming file 5362
Transforming file 5363
Transforming file 5364
Transforming file 5365
Transforming file 5366
Transforming file 5367
Transforming file 5368
Transforming file 5369
Transforming file 5370
Transforming file 5371
Transforming file 5372
Transforming file 5373
Transforming file 5374
Transforming file 5375
Transforming file 5376
Transforming file 5377
Transforming file 5378
Transforming file 5379
Transforming file 5380
Transforming file 5381
Transforming file 5382
Transforming file 5383
Transforming file 5384
Transforming file 5385
Transformin

Transforming file 5701
Transforming file 5702
Transforming file 5703
Transforming file 5704
Transforming file 5705
Transforming file 5706
Transforming file 5707
Transforming file 5708
Transforming file 5709
Transforming file 5710
Transforming file 5711
Transforming file 5712
Transforming file 5713
Transforming file 5714
Transforming file 5715
Transforming file 5716
Transforming file 5717
Transforming file 5718
Transforming file 5719
Transforming file 5720
Transforming file 5721
Transforming file 5722
Transforming file 5723
Transforming file 5724
Transforming file 5725
Transforming file 5726
Transforming file 5727
Transforming file 5728
Transforming file 5729
Transforming file 5730
Transforming file 5731
Transforming file 5732
Transforming file 5733
Transforming file 5734
Transforming file 5735
Transforming file 5736
Transforming file 5737
Transforming file 5738
Transforming file 5739
Transforming file 5740
Transforming file 5741
Transforming file 5742
Transforming file 5743
Transformin

Transforming file 6058
Transforming file 6059
Transforming file 6060
Transforming file 6061
Transforming file 6062
Transforming file 6063
Transforming file 6064
Transforming file 6065
Transforming file 6066
Transforming file 6067
Transforming file 6068
Transforming file 6069
Transforming file 6070
Transforming file 6071
Transforming file 6072
Transforming file 6073
Transforming file 6074
Transforming file 6075
Transforming file 6076
Transforming file 6077
Transforming file 6078
Transforming file 6079
Transforming file 6080
Transforming file 6081
Transforming file 6082
Transforming file 6083
Transforming file 6084
Transforming file 6085
Transforming file 6086
Transforming file 6087
Transforming file 6088
Transforming file 6089
Transforming file 6090
Transforming file 6091
Transforming file 6092
Transforming file 6093
Transforming file 6094
Transforming file 6095
Transforming file 6096
Transforming file 6097
Transforming file 6098
Transforming file 6099
Transforming file 6100
Transformin

Transforming file 6415
Transforming file 6416
Transforming file 6417
Transforming file 6418
Transforming file 6419
Transforming file 6420
Transforming file 6421
Transforming file 6422
Transforming file 6423
Transforming file 6424
Transforming file 6425
Transforming file 6426
Transforming file 6427
Transforming file 6428
Transforming file 6429
Transforming file 6430
Transforming file 6431
Transforming file 6432
Transforming file 6433
Transforming file 6434
Transforming file 6435
Transforming file 6436
Transforming file 6437
Transforming file 6438
Transforming file 6439
Transforming file 6440
Transforming file 6441
Transforming file 6442
Transforming file 6443
Transforming file 6444
Transforming file 6445
Transforming file 6446
Transforming file 6447
Transforming file 6448
Transforming file 6449
Transforming file 6450
Transforming file 6451
Transforming file 6452
Transforming file 6453
Transforming file 6454
Transforming file 6455
Transforming file 6456
Transforming file 6457
Transformin

...

Transforming file 7129
Transforming file 7130
Transforming file 7131
Transforming file 7132
Transforming file 7133
Transforming file 7134
Transforming file 7135
Transforming file 7136
Transforming file 7137
Transforming file 7138
Transforming file 7139
Transforming file 7140
Transforming file 7141
Transforming file 7142
Transforming file 7143
Transforming file 7144
Transforming file 7145
Transforming file 7146
Transforming file 7147
Transforming file 7148
Transforming file 7149
Transforming file 7150
Transforming file 7151
Transforming file 7152
Transforming file 7153
Transforming file 7154
Transforming file 7155
Transforming file 7156
Transforming file 7157
Transforming file 7158
Transforming file 7159
Transforming file 7160
Transforming file 7161
Transforming file 7162
Transforming file 7163
Transforming file 7164
Transforming file 7165
Transforming file 7166
Transforming file 7167
Transforming file 7168
Transforming file 7169
Transforming file 7170
Transforming file 7171
Transformin

Transforming file 7486
Transforming file 7487
Transforming file 7488
Transforming file 7489
Transforming file 7490
Transforming file 7491
Transforming file 7492
Transforming file 7493
Transforming file 7494
Transforming file 7495
Transforming file 7496
Transforming file 7497
Transforming file 7498
Transforming file 7499
Transforming file 7500
Transforming file 7501
Transforming file 7502
Transforming file 7503
Transforming file 7504
Transforming file 7505
Transforming file 7506
Transforming file 7507
Transforming file 7508
Transforming file 7509
Transforming file 7510
Transforming file 7511
Transforming file 7512
Transforming file 7513
Transforming file 7514
Transforming file 7515
Transforming file 7516
Transforming file 7517
Transforming file 7518
Transforming file 7519
Transforming file 7520
Transforming file 7521
Transforming file 7522
Transforming file 7523
Transforming file 7524
Transforming file 7525
Transforming file 7526
Transforming file 7527
Transforming file 7528
Transformin

Transforming file 7843
Transforming file 7844
Transforming file 7845
Transforming file 7846
Transforming file 7847
Transforming file 7848
Transforming file 7849
Transforming file 7850
Transforming file 7851
Transforming file 7852
Transforming file 7853
Transforming file 7854
Transforming file 7855
Transforming file 7856
Transforming file 7857
Transforming file 7858
Transforming file 7859
Transforming file 7860
Transforming file 7861
Transforming file 7862
Transforming file 7863
Transforming file 7864
Transforming file 7865
Transforming file 7866
Transforming file 7867
Transforming file 7868
Transforming file 7869
Transforming file 7870
Transforming file 7871
Transforming file 7872
Transforming file 7873
Transforming file 7874
Transforming file 7875
Transforming file 7876
Transforming file 7877
Transforming file 7878
Transforming file 7879
Transforming file 7880
Transforming file 7881
Transforming file 7882
Transforming file 7883
Transforming file 7884
Transforming file 7885
Transformin

Transforming file 8200
Transforming file 8201
Transforming file 8202
Transforming file 8203
Transforming file 8204
Transforming file 8205
Transforming file 8206
Transforming file 8207
Transforming file 8208
Transforming file 8209
Transforming file 8210
Transforming file 8211
Transforming file 8212
Transforming file 8213
Transforming file 8214
Transforming file 8215
Transforming file 8216
Transforming file 8217
Transforming file 8218
Transforming file 8219
Transforming file 8220
Transforming file 8221
Transforming file 8222
Transforming file 8223
Transforming file 8224
Transforming file 8225
Transforming file 8226
Transforming file 8227
Transforming file 8228
Transforming file 8229
Transforming file 8230
Transforming file 8231
Transforming file 8232
Transforming file 8233
Transforming file 8234
Transforming file 8235
Transforming file 8236
Transforming file 8237
Transforming file 8238
Transforming file 8239
Transforming file 8240
Transforming file 8241
Transforming file 8242
Transformin

Transforming file 8557
Transforming file 8558
Transforming file 8559
Transforming file 8560
Transforming file 8561
Transforming file 8562
Transforming file 8563
Transforming file 8564
Transforming file 8565
Transforming file 8566
Transforming file 8567
Transforming file 8568
Transforming file 8569
Transforming file 8570
Transforming file 8571
Transforming file 8572
Transforming file 8573
Transforming file 8574
Transforming file 8575
Transforming file 8576
Transforming file 8577
Transforming file 8578
Transforming file 8579
Transforming file 8580
Transforming file 8581
Transforming file 8582
Transforming file 8583
Transforming file 8584
Transforming file 8585
Transforming file 8586
Transforming file 8587
Transforming file 8588
Transforming file 8589
Transforming file 8590
Transforming file 8591
Transforming file 8592
Transforming file 8593
Transforming file 8594
Transforming file 8595
Transforming file 8596
Transforming file 8597
Transforming file 8598
Transforming file 8599
Transformin